# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

* In this capstone project I developed a data pipeline that creates an analytics database for querying information about immigration into the U.S. The analytics tables are hosted in a Redshift Database and the pipeline implementation was done using Apache Airflow.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [51]:
# Do all imports and installs here
import pandas as pd
pd.set_option('display.max_columns', None)
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
import util
import os

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

* Scope of the project is to create an ETL pipeline for processing, cleaning and storing data related to US I94 immigration data, and country codes.

* Output of the ETL pipeline: processed data stored in Star schema model to parquet files. Tools: python, pandas, pyspark

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The following datasets were used to create the analytics database:

* I94 Immigration Data: Each report contains international visitor arrival statistics by world regions and select countries (including top 20), type of visa, mode of transportation, age groups, states visited (first intended address only), and the top ports of entry (for select countries). This data comes from the US National Tourism and Trade Office found [here](https://travel.trade.gov/research/reports/i94/historical/2016.html).

* World Temperature Data: This dataset came from Kaggle found [here](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).

* U.S. City Demographic Data: This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000. Dataset comes from OpenSoft found [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).

* Airport Code Table: This is a simple table of airport codes and corresponding cities. The airport codes may refer to either IATA airport code, a three-letter code which is used in passenger reservation, ticketing and baggage-handling systems, or the ICAO airport code which is a four letter code used by ATC systems and for airports that do not have an IATA airport code (from wikipedia). It comes from [here](https://datahub.io/core/airport-codes#data).

In [2]:
# Read in the data here

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11") \
        .enableHiveSupport().getOrCreate()

df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [4]:
%%time
df_spark.count()

CPU times: user 5.81 ms, sys: 916 µs, total: 6.72 ms
Wall time: 39.2 s


3096313

In [5]:
#write to parquet
# df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

In [6]:
%%time
print(df_spark.count())

3096313
CPU times: user 1.03 ms, sys: 160 µs, total: 1.19 ms
Wall time: 876 ms


#### Immigration Data

In [7]:
print(df_spark.count())
df_spark.printSchema()
df_spark.limit(2).toPandas()

3096313
root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nul

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5748517.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,40.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1976.0,10292016,F,None,QF,9.495387e+10,00011,B1
1,5748518.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,32.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1984.0,10292016,F,None,VA,9.495562e+10,00007,B1


#### Airports Data

In [8]:
airportcodes_df = spark.read.csv("../workspace//airport-codes_csv.csv",
                                 header = True)
print(airportcodes_df.count())
airportcodes_df.printSchema()
airportcodes_df.limit(2).toPandas()

55075
root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"


#### U.S. City Demographic Data

In [9]:
demographic_df = spark.read.csv("../workspace/us-cities-demographics.csv",
                                sep = ';',
                                header = True)
print(demographic_df.count())
demographic_df.printSchema()
demographic_df.limit(2).toPandas()

2891
root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.6,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723


#### Immigration Sample Data

In [10]:
immigration_df = spark.read.csv("../workspace/immigration_data_sample.csv",
                                header = True)
print(immigration_df.count())
immigration_df.printSchema()
immigration_df.limit(2).toPandas()

1000
root
 |-- _c0: string (nullable = true)
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable 

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,None,None,G,O,None,M,1955.0,07202016,F,None,JL,56582674633.0,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,None,G,R,None,M,1990.0,10222016,M,None,*GA,94361995930.0,XBLNG,B2


#### Global Temperature Data

In [11]:
temperature_df = spark.read.csv("../../data2/GlobalLandTemperaturesByCity.csv",
                                 header = True)
print(temperature_df.count())
temperature_df.printSchema()
temperature_df.limit(2).toPandas()

8599212
root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.7369999999999999,Århus,Denmark,57.05N,10.33E
1,1743-12-01,None,None,Århus,Denmark,57.05N,10.33E


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [12]:
# Perform quality checks here
def data_quality_check(dataframe, target):
    df_count = dataframe.count()
    df_na_count = dataframe.dropna(how="any").count()
    df_dup_count = dataframe.dropDuplicates().count()

    print(f"Number of records with missing data in {target} dataset: {df_count - df_na_count}")
    print(f"Number of records with duplicate data in {target} dataset: {df_count - df_dup_count}")

In [13]:
data_quality_check(immigration_df, 'Immigration')

Number of records with missing data in Immigration dataset: 1000
Number of records with duplicate data in Immigration dataset: 0


In [14]:
data_quality_check(temperature_df, 'Temperature')

Number of records with missing data in Temperature dataset: 364130
Number of records with duplicate data in Temperature dataset: 0


In [15]:
data_quality_check(airportcodes_df, 'Airportcodes')

Number of records with missing data in Airportcodes dataset: 52329
Number of records with duplicate data in Airportcodes dataset: 0


In [16]:
data_quality_check(demographic_df, 'Demographic')

Number of records with missing data in Demographic dataset: 16
Number of records with duplicate data in Demographic dataset: 0


In [17]:
# Clean data here
immigration_df = immigration_df.na.fill({'i94mode': 0.0, 'i94addr': 'NA','depdate': 0.0, 'i94bir': 'NA', \
                        'i94visa': 0.0, 'count': 0.0, 'dtadfile': 'NA', 'visapost': 'NA', \
                        'occup': 'NA', 'entdepa': 'NA', 'entdepd': 'NA', 'entdepu': 'NA', \
                        'matflag': 'NA', 'biryear': 0.0, 'dtaddto': 'NA', 'gender': 'NA', \
                        'insnum': 'NA', 'airline': 'NA', 'admnum': 0.0, 'fltno': 'NA', 'visatype': 'NA'})

In [18]:
data_quality_check(immigration_df, 'Immigration')

Number of records with missing data in Immigration dataset: 0
Number of records with duplicate data in Immigration dataset: 0


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
* Map out the conceptual data model and explain why you chose that model
    * The Data Model that will be used is a star schema. This is because since there will be so much data in the data warehouse, it is important to simplify the joins as not doing so will significantly slow down queries.
        * The us immigration data will serve as the fact table since that is the primary event in question i.e people immigrating into the US. 
        * The demographic data will be a dimension table and should join with the immigration data via the city name. 
        * The Tempurature data will be another dimension table which can join with the immigration data based on the date of arrival. 
        * The airport codes can be joined based on the city name.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
* Currently the data exists as csv files and the end goal is to have a datawarehouse that can be quried. AWS Will be used for the pipeline and the steps are as follows:
    1. extract the source data csv files using spark and examine and report and the data quality
    2. clean the data as per the above cleaning transformation requirements
    3. set up AWS using IAC and create the needed S3 buckets for the data lake and the tables needed for redshift
    4. load the cleaned spark dataframes into S3 as csv files
    5. use S3 to load data into redshift

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [54]:
# Write code here
def create_demographics_dimension_table(dataframe, output_path):
    df = dataframe.withColumnRenamed('Median Age','median_age') \
            .withColumnRenamed('Male Population', 'male_population') \
            .withColumnRenamed('Female Population', 'female_population') \
            .withColumnRenamed('Total Population', 'total_population') \
            .withColumnRenamed('Number of Veterans', 'number_of_veterans') \
            .withColumnRenamed('Foreign-born', 'foreign_born') \
            .withColumnRenamed('Average Household Size', 'average_household_size') \
            .withColumnRenamed('State Code', 'state_code')
    # lets add an id column
    df = df.withColumn('id', F.monotonically_increasing_id())
    
    # write dimension to parquet file
    df.write.parquet(os.path.join(output_path, 'demographics'), mode='overwrite', partitionBy=['City', 'State'])
    
    return df

In [55]:
output_path = 'output'
create_demographics_dimension_table(demographic_df, output_path)

DataFrame[City: string, State: string, median_age: string, male_population: string, female_population: string, total_population: string, number_of_veterans: string, foreign_born: string, average_household_size: string, state_code: string, Race: string, Count: string, id: bigint]

In [56]:
!pwd

/home/workspace


In [57]:
# !rm -r /home/workspace/output/demographics/

In [21]:
temperature_df.limit(1).toPandas()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.7369999999999999,Århus,Denmark,57.05N,10.33E


In [22]:
airportcodes_df.limit(1).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"


In [43]:
demographic_df.count()

2891

In [42]:
demographic_df.limit(1).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.6,MD,Hispanic or Latino,25924


In [20]:
immigration_df.limit(1).toPandas()

,_c0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NA,NA,G,O,NA,M,1955.0,07202016,F,NA,JL,56582674633.0,00782,WT


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [49]:
# Perform quality checks here
def quality_checks(df, table_name):
    total_count = df.count()

    if total_count == 0:
        print(f"Data quality check failed for {table_name} with zero records!")
    else:
        print(f"Data quality check passed for {table_name} with {total_count:,} records.")
    return

In [50]:
quality_checks(immigration_df, 'immigration')

Data quality check passed for immigration with 1,000 records.


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

##### Fact Table

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">record_id</td><td class="tg-0pky">Unique record ID</td></tr>
 <tr><td class="tg-0pky">country_residence_code</td><td class="tg-0pky">3 digit code for immigrant country of residence </td></tr>    
 <tr><td class="tg-0pky">visa_type_key</td><td class="tg-0pky">A numerical key that links to the visa_type dimension table</td></tr>
 <tr><td class="tg-0pky">state_code</td><td class="tg-0pky">US state of arrival</td></tr>
 <tr><td class="tg-0pky">i94yr</td><td class="tg-0pky">4 digit year</td></tr>
 <tr><td class="tg-0pky">i94mon</td><td class="tg-0pky">Numeric month</td></tr>
 <tr><td class="tg-0pky">i94port</td><td class="tg-0pky">Port of admission</td></tr>
 <tr><td class="tg-0pky">arrdate</td><td class="tg-0pky">Arrival Date in the USA</td></tr>
 <tr><td class="tg-0pky">i94mode</td><td class="tg-0pky">Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported)</td></tr>
 <tr><td class="tg-0pky">i94addr</td><td class="tg-0pky">USA State of arrival</td></tr>
 <tr><td class="tg-0pky">depdate</td><td class="tg-0pky">Departure Date from the USA</td></tr>
 <tr><td class="tg-0pky">i94bir</td><td class="tg-0pky">Age of Respondent in Years</td></tr>
 <tr><td class="tg-0pky">i94visa</td><td class="tg-0pky">Visa codes collapsed into three categories</td></tr>
 <tr><td class="tg-0pky">count</td><td class="tg-0pky">Field used for summary statistics</td></tr>
 <tr><td class="tg-0pky">dtadfile</td><td class="tg-0pky">Character Date Field - Date added to I-94 Files</td></tr>
 <tr><td class="tg-0pky">visapost</td><td class="tg-0pky">Department of State where where Visa was issued </td></tr>
 <tr><td class="tg-0pky">occup</td><td class="tg-0pky">Occupation that will be performed in U.S</td></tr>
 <tr><td class="tg-0pky">entdepa</td><td class="tg-0pky">Arrival Flag - admitted or paroled into the U.S.</td></tr>
 <tr><td class="tg-0pky">entdepd</td><td class="tg-0pky">Departure Flag - Departed, lost I-94 or is deceased</td></tr>
 <tr><td class="tg-0pky">entdepu</td><td class="tg-0pky">Update Flag - Either apprehended, overstayed, adjusted to perm residence</td></tr>
 <tr><td class="tg-0pky">matflag</td><td class="tg-0pky">Match flag - Match of arrival and departure records</td></tr>
 <tr><td class="tg-0pky">biryear</td><td class="tg-0pky">4 digit year of birth</td></tr>
 <tr><td class="tg-0pky">dtaddto</td><td class="tg-0pky">Character Date Field - Date to which admitted to U.S. (allowed to stay until)</td></tr>
 <tr><td class="tg-0pky">gender</td><td class="tg-0pky">Non-immigrant sex</td></tr>
</table>

##### Country Dimension Table
<p>  
<i>The country code and country_name fields come from the labels description SAS file while the average_temperature data comes from the global land temperature by cities data.</i>
</p>
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">country_code</td><td class="tg-0pky">Unique country code</td></tr>
 <tr><td class="tg-0pky">country_name</td><td class="tg-0pky">Name of country</td></tr>    
 <tr><td class="tg-0pky">average_temperature</td><td class="tg-0pky">Average temperature of country</td></tr>
</table>

 ##### Immigration Calendar Dimension Table
<p>
<i>The whole of this dataset comes from the immigration_data_sample dataset.</i>
</p>
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">id</td><td class="tg-0pky">Unique id</td></tr>
 <tr><td class="tg-0pky">arrdate</td><td class="tg-0pky">Arrival date into US</td></tr>    
 <tr><td class="tg-0pky">arrival_year</td><td class="tg-0pky">Arrival year into US</td></tr>
 <tr><td class="tg-0pky">arrival_month</td><td class="tg-0pky">Arrival MonthS</td></tr>
 <tr><td class="tg-0pky">arrival_day</td><td class="tg-0pky">Arrival Day</td></tr>
 <tr><td class="tg-0pky">arrival_week</td><td class="tg-0pky">Arrival Week</td></tr>
 <tr><td class="tg-0pky">arrival_weekday</td><td class="tg-0pky">Arrival WeekDay</td></tr>
</table>

##### US Demographics Dimension Table
<p>
<i>The whole of this dataset comes from the us-cities-demographics data.</i>
</p>

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">id</td><td class="tg-0pky">Record id</td>
 <tr><td class="tg-0pky">state_code</td><td class="tg-0pky">US state code </td>
 <tr><td class="tg-0pky">City</td><td class="tg-0pky">City Name</td>
 <tr><td class="tg-0pky">State</td><td class="tg-0pky">US State where city is located</td>
 <tr><td class="tg-0pky">Median Age</td><td class="tg-0pky">Median age of the population</td>
 <tr><td class="tg-0pky">Male Population</td><td class="tg-0pky">Count of male population</td>
 <tr><td class="tg-0pky">Female Population</td><td class="tg-0pky">Count of female population</td>
 <tr><td class="tg-0pky">Total Population</td><td class="tg-0pky">Count of total population</td>
 <tr><td class="tg-0pky">Number of Veterans</td><td class="tg-0pky">Count of total Veterans</td>
 <tr><td class="tg-0pky">Foreign born</td><td class="tg-0pky">Count of residents of the city that were not born in the city</td>
 <tr><td class="tg-0pky">Average Household Size</td><td class="tg-0pky">Average city household size</td>
 <tr><td class="tg-0pky">Race</td><td class="tg-0pky">Respondent race</td>
 <tr><td class="tg-0pky">Count</td><td class="tg-0pky">Count of city's individual per race</td>
</table>

#### Step 5: Complete Project Write Up

* Clearly state the rationale for the choice of tools and technologies for the project.
    * Apache spark was used because of:
        * it's ability to handle multiple file formats with large amounts of data. 
        * Spark has easy-to-use APIs for operating on large datasets
        * Apache Spark offers a lightning-fast unified analytics engine for big data.

* Propose how often the data should be updated and why.
    * The data will be updated monthly, since the current I94 immigration data is updated monthly.

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
     * Spark and AWS EMR can handle the increase but we would consider increasing the number of nodes in our cluster.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
     * Apache Airflow will be used to schedule and run data pipelines.
 * The database needed to be accessed by 100+ people.
     * We would move our analytics database into Amazon Redshift. Redshift configuration can be changed to make it more available. Some examples of that would be having Redshift servicing multiple availability zones. In addition, if there are common workloads or queries that need to be run by many of these people, they can be prepared in advanced such as pre aggregated OLAP cubes.